In [3]:
# default_exp core

# module name here

> API details.

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

[nb_2_html](https://github.com/BNIA/dataplay/blob/main/notebooks/04_nb_2_html.ipynb)

In [4]:
#export
import re
from IPython.display import clear_output
import marko

In [5]:
#export
from nbdev import export2html
from nbdev.export2html import read_nb, get_metadata, clean_exports
from nbdev.export import split_flags_and_code, check_re_multi

In [59]:
#export
# 1st fn. passes each cell to decision fn.
def convertNb(nb):  return [cleanCell(c) for c in nb['cells']]

# 2nd fn. returns text or passes cell to 'code cell' processor
def cleanCell(cell): return marko.convert(cell.source) if cell.cell_type == 'markdown' else processCode(cell)

# 3rd fn. Calls getFlags, processSource, processOutput
def processCode(cell):
  x = [cell.source]
  flags = getFlags(x[0])
  x[0] = processSource( x[0], flags )
  if ( len(cell.outputs) ):
    for o in cell.outputs: x.append( processOutput(o, flags) )
  return x

In [7]:
#export
# Detect and stripout and handle flags.
def getFlags(source):
  input_aug = ['#collapse_input_open', '#collapse_input', '#collapse_output_open', '#collapse_output',
               '#hide_input', '#hide_output', '#hide ', '%%capture', '%%javascript', '%%html']
  return [ x for x in input_aug if re.search(x, source) ]

# enclose the text within a collapsable html drawer.
def makeDetails( content, open ):
  return "<details "+('open' if open else '')+"> <summary>Click to toggle</summary> "+content+"</details>"

In [8]:
#export
# Strip Flags from text, make details, hide all.
def processSource (source, flags):
  for lbl in flags:
    source = source.replace(lbl+'\r\n', "")
    source = source.replace(lbl+'\n', "") # Strip the Flag
    if (lbl == '#collapse_input_open'): source = makeDetails( source, True )
    if (lbl == '#collapse_input'): source = makeDetails( source, False )
    if (lbl == '#hide '): source = ''
    if (lbl == '#hide_input'): source = ''
    if (lbl == '%%javascript'): source = ''
    if (lbl == '%%html'): source = ''
    if (lbl == '%%capture'): source = ''
  return source

In [60]:
#export
def processOutput(source, flags):
  if(source.output_type=='error'):  return ''
  if(source.output_type=='stream'):
    if(source.name=='stderr'): return ''
    source.data = {'text/html': source.text} # This will have the stream process as text/html.
  print(source.data.keys())
  if ( hasattr(source.data, 'text/html') ): source=source.data['text/html'];
  elif ( hasattr(source.data, 'application/javascript') ): source='<script>'+source.data['application/javascript']+'</script>'
  elif ( hasattr(source.data, 'image/png') ): source= "<img src=\"data:image/png;base64," + source.data['image/png'] + "\" alt='Image Alt Text'>"
  elif ( hasattr(source.data, 'text/plain') ): source = '' if re.search("<Figure", source.data['text/plain']) else source.data['text/plain'];
  # print(source)
  for lbl in flags:
    source = source.replace(lbl+'\r\n', "")
    source = source.replace(lbl+'\n', "") # Strip the Flag
    if (lbl == '#collapse_output_open'): source = makeDetails( source, True )
    if (lbl == '#collapse_output'): source = makeDetails( source, False )
    if (lbl == '#hide_output'): source = ''
    if (lbl == '#hide '): source = ''
  return source
  #output_type == 'stream' ==> text
  #output_type == 'display_data' ==> data{'application/javascript' or 'text/html' or 'execute_result'}

In [ ]:
ls

In [ ]:
nb = read_nb('index.ipynb')
outp = convertNb(nb)

In [66]:
#export
def runit(fname='index.ipynb'): 
  nb = read_nb(fname)
  meta_jekyll = get_metadata(nb['cells'])
  outp = convertNb(nb)
  txt = ''.join( [''.join(c) for c in outp] )
  return txt

In [13]:
t = runit()

In [67]:
t

"<p>Hi there; It looks like you found my blog! </p>\n<p>I'd like to write about my day to day adventures here.</p>\n"

In [ ]:
def templatize(txt="text123", template='article'):
    template = open("../docs/_templates/"+template+".html", "r").read()
    return template

templatize()

In [126]:
ls

000_core.ipynb        002_Parcels.ipynb     index.ipynb
001a_core_test.ipynb  003_Guestbooks.ipynb  pivottablejs.html
001b_core_test.ipynb  013_CV_Misc.ipynb
001c_Example.ipynb    _devserver.ipynb


[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/CVMiniGames/CVMiniGames.github.io/main)
[![Binder](https://pete88b.github.io/fastpages/assets/badges/colab.svg)](https://colab.research.google.com/github/CVMiniGames/CVMiniGames.github.io/blob/main/blog/notebooks/{{filename}}.ipynb)
[![Binder](https://pete88b.github.io/fastpages/assets/badges/github.svg)](https://github.com/CVMiniGames/CVMiniGames.github.io/blob/main/blog/notebooks/{{filename}}.ipynb) 
[![GitHub last commit](https://img.shields.io/github/last-commit/CVMiniGames/CVMiniGames.github.io.svg?style=flat)]()  

[![GitHub stars](https://img.shields.io/github/stars/CVMiniGames/CVMiniGames.github.io.svg?style=social&label=Star)](https://github.com/CVMiniGames/CVMiniGames.github.io) 
[![GitHub watchers](https://img.shields.io/github/watchers/CVMiniGames/CVMiniGames.github.io.svg?style=social&label=Watch)](https://github.com/CVMiniGames/CVMiniGames.github.io) 
[![GitHub forks](https://img.shields.io/github/forks/CVMiniGames/CVMiniGames.github.io.svg?style=social&label=Fork)](https://github.com/CVMiniGames/CVMiniGames.github.io) 
[![GitHub followers](https://img.shields.io/github/followers/CVMiniGames.svg?style=social&label=Follow)](https://github.com/CVMiniGames/CVMiniGames.github.io) 

[![Tweet](https://img.shields.io/twitter/url/https/github.com/CVMiniGames/CVMiniGames.github.io.svg?style=social)](https://twitter.com/intent/tweet?text=Check%20out%20these%20%E2%9C%A8%20colab%20by%20@CVMiniGames%20https://github.com/CVMiniGames/CVMiniGames.github.io) 
[![Twitter Follow](https://img.shields.io/twitter/follow/CVMiniGames.svg?style=social)](https://twitter.com/CVMiniGames)

Viewcount: ![Visitors](https://visitor-badge.laobi.icu/badge?page_id=CVMiniGames.CVMiniGames.github.io)

In [175]:
#export
# Must be in directory of ipynb you want to convert to html.
def publish(fname='index'): 
  sys.path.pop()
  sys.path.append('workspaces/3Diot.github.io/docs/')
  from blog import core 
  nb = read_nb(fname+'.ipynb')
  meta_jekyll = get_metadata(nb['cells'])
  outp = convertNb(nb)
  outp = ''.join( [''.join(c) for c in outp] ) 
  resp = ''.join( [''.join(c) for c in outp] )
  template = open("../docs/_templates/"+meta_jekyll['template']+".html", "r").read() 
  template = template.replace('{{title}}', meta_jekyll['title']) 
  template = template.replace('{{filename}}', fname+'.ipynb') 
  txt = template.replace('{{content}}', resp ) 
  f = open("../docs/"+meta_jekyll['title'].lower().replace(' ', '_')+".html", "w")
  f.write(txt)
  f.close()
  return txt
  # return "<p>Hello, Worrrld!</p>"

In [176]:
publish('002_Parcels')

dict_keys(['text/html', 'text/plain'])
dict_keys(['text/html'])
dict_keys(['text/html', 'text/plain'])


'<!DOCTYPE html>\n<html lang="en">\n<head>\n  <meta charset="UTF-8">\n  <meta name="viewport" content="width=device-width, initial-scale=1.0">\n  <title>Parcels</title>\n  <script type="module" src="https://cdn.jsdelivr.net/npm/@ionic/core/dist/ionic/ionic.esm.js"></script>\n  <script nomodule src="https://cdn.jsdelivr.net/npm/@ionic/core/dist/ionic/ionic.js"></script>\n  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/@ionic/core/css/ionic.bundle.css" />\n</head>\n<body>\n  <ion-app>\n    <ion-header translucent > <!-- class="ion-no-border" -->\n      <ion-toolbar>\n        <ion-title>Parcels</ion-title>\n      </ion-toolbar> \n    </ion-header>,\n    <ion-content fullscreen class="ion-padding"> \n      <!-- \n        # https://github.com/ionic-team/ionic-docs/blob/legacy/src/demos/api/content/index.html \n        # https://github.com/ionic-team/ionic-docs/blob/legacy/src/demos/api/infinite-scroll/index.html\n        # https://github.com/ionic-team/ionic-docs/blob/legacy/src

In [155]:
IPython.display.HTML( "<p>Hi there; It looks like you found my blog! </p>\n<p>I'd like to write about my day to day adventures here.</p>\n" )